# Global Flood Monitoring – STAC → GIF/MP4

This notebook walks you through the full workflow to:
1. Query **GFM (Global Flood Monitoring)** items from a STAC API for a chosen **Area of Interest (AOI)** and **time range**.
2. Load rasters with **ODC-STAC**, reproject to **EPSG:4326**, and export per-day **PNG overlays** (blue flood masks).
   
~~3. Build an **interactive ipyleaflet** viewer with a play/slider control and a simple legend (fading cumulative floods).~~

4. Export a high-quality **GIF and MP4** video with:
   - OSM basemap (downloaded XYZ tiles once and reprojected to AOI),
   - cumulative flood overlay (with fading effect),
   - a large **bar chart** showing flooded area (km²) per day,
   - a progress bar & static legend that **does not flicker**.

> You need: `pystac-client`, `odc-stac`, `rioxarray`, `xarray`, `ipywidgets`, `ipyleaflet`, `rasterio`, `matplotlib`, `imageio`, `Pillow`. For MP4 export, `ffmpeg` on PATH.


## 0) (Optional) Install dependencies

Uncomment and run if you need to install missing packages. Restart the kernel afterward.


In [1]:
# %%bash
# pip install --upgrade pip
# pip install pystac-client odc-stac rioxarray xarray ipywidgets ipyleaflet rasterio pillow imageio matplotlib tqdm
# # Enable classic widgets (if using classic notebook)
# jupyter nbextension enable --py widgetsnbextension


## 1) Imports & configuration

We import geospatial and visualization libraries and set basic paths.


In [2]:
from shapely.geometry import box, mapping
from pystac_client import Client
from odc import stac as odc_stac
import pyproj
import rioxarray
import xarray as xr
from tqdm.notebook import tqdm
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import imageio.v2 as imageio
import matplotlib.image as mpimg
import os, io, re, math, warnings, subprocess, requests, shutil, base64
from datetime import datetime
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.patches import FancyBboxPatch
import rasterio
from rasterio.transform import from_bounds, Affine
from rasterio.warp import reproject, Resampling
from IPython.display import display
import ipywidgets as widgets
from ipyleaflet import Map, ImageOverlay, basemaps, WidgetControl


# Handy switches and paths
FLOOD_DIR = "flood_frames"
STAC_URL  = "https://stac.eodc.eu/api/v1"
COLLECTION_ID = "GFM"

os.makedirs(FLOOD_DIR, exist_ok=True)


## 2) Define AOI and time range

Adjust the AOI and the analysis window as needed.


In [3]:
# Pakistan (smaller area) – adjust as needed
aoi_geometry = box(73.162312, 31.098403, 74.601359, 32.174052)

# Time window for flood monitoring
time_range = ("2025-06-27", "2025-07-17")  # (YYYY-MM-DD, YYYY-MM-DD)
print("AOI bounds:", aoi_geometry.bounds)
print("Time range:", time_range)


AOI bounds: (73.162312, 31.098403, 74.601359, 32.174052)
Time range: ('2025-06-27', '2025-07-17')


## 3) Query the STAC API

Search the **GFM** collection for items intersecting the AOI within the time window.


In [4]:
client = Client.open(STAC_URL)

search = client.search(
    collections=[COLLECTION_ID],
    intersects=mapping(aoi_geometry),
    datetime=f"{time_range[0]}/{time_range[1]}"
)

items = search.item_collection()
print(f"🔍 Found {len(items)} items.")
if len(items) == 0:
    raise RuntimeError("No STAC items found for the given AOI/time range.")


🔍 Found 117 items.


## 4) Load rasters via ODC-STAC

We extract source CRS/GSD from the first item, load `ensemble_flood_extent`, and reproject to **EPSG:4326** for visualization and bounds.


In [5]:

# Extract CRS and nominal resolution (gsd) from the first item
crs = pyproj.CRS.from_wkt(items[0].properties["proj:wkt2"])
resolution = items[0].properties.get("gsd", None)
print("Source CRS:", crs)
print("GSD (m):", resolution)

# Desired band
bands = ["ensemble_flood_extent"]

# Load cube (fail_on_error=False nech to nepadne na jednom súbore)
xx = odc_stac.load(
    items,
    crs=crs,
    bbox=aoi_geometry.bounds,
    bands=bands,
    resolution=resolution,
    dtype="uint8",
    fail_on_error=False,
)

# Reproject to WGS84 for visualization / bounds extraction
xx = xx.rio.reproject("EPSG:4326")


Source CRS: PROJCS["Azimuthal_Equidistant",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["false_easting",4340913.84808],PARAMETER["false_northing",4812712.92347],PARAMETER["longitude_of_center",94.0],PARAMETER["latitude_of_center",47.0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]
GSD (m): 20.0


## 5) Build binary flood masks and export PNG overlays

We treat values **not equal to 0 or 255** as flooded. For each timestep we export a transparent-blue PNG to `flood_frames/`. We also export a max-extent GeoTIFF for convenience.


In [6]:
# Binary flood: 1 if value is NOT 0 or 255, else 0
flood_binary = xr.where((xx.ensemble_flood_extent != 255) & (xx.ensemble_flood_extent != 0), 1, 0)

# Optional: max flood extent across time (as GeoTIFF in EPSG:4326)
flood_max = xr.where(flood_binary.sum(dim="time") > 0, 1, 0).astype("uint8")
flood_max.rio.to_raster("max_flood_extent_epsg4326.tif", compress="LZW")

# Clean up existing frames (if any)
for f in os.listdir(FLOOD_DIR):
    p = os.path.join(FLOOD_DIR, f)
    if os.path.isfile(p):
        os.remove(p)

# Compute all flood rasters (if lazy)
flood_data = xx.ensemble_flood_extent.compute()

# Export each timestep as RGBA PNG (blue channel + alpha = 255 where flood)
for i, t in enumerate(tqdm(flood_data.time.values, desc="Export PNG frames")):
    day = str(np.datetime_as_string(t, unit='D'))
    flood_img = flood_data.sel(time=t)
    flood_bin = xr.where((flood_img != 255) & (flood_img != 0), 255, 0).astype("uint8")

    vals = flood_bin.values
    rgba = np.zeros((*vals.shape, 4), dtype=np.uint8)
    rgba[..., 2] = vals  # Blue
    rgba[..., 3] = vals  # Alpha

    out_path = os.path.join(FLOOD_DIR, f"flood_{i:02d}_{day}.png")
    imageio.imwrite(out_path, rgba)

print("✅ PNG frames saved to:", FLOOD_DIR)


Export PNG frames:   0%|          | 0/67 [00:00<?, ?it/s]

✅ PNG frames saved to: flood_frames


## 6) Prepare cumulative overlays (fading) and bounds

We read the generated PNGs, build cumulative overlays with a fading effect (older floods are lighter), and compute bounds from the reprojected raster.


In [7]:
# Read back overlays & labels
file_list = sorted([f for f in os.listdir(FLOOD_DIR) if f.lower().endswith((".png", ".jpg", ".jpeg"))])
labels = []
binary_masks = []

expected_shape = None
for fname in file_list:
    img = mpimg.imread(os.path.join(FLOOD_DIR, fname))
    mask = img[..., 2] > 0  # blue channel > 0 -> flooded

    if np.sum(mask) == 0:
        continue

    if expected_shape is None:
        expected_shape = mask.shape

    if mask.shape != expected_shape:
        print(f"❌ Skipping {fname} – different shape: {mask.shape}")
        continue

    binary_masks.append(mask)
    labels.append(fname.replace("flood_", "").replace(".png", ""))

# Bounds from the reprojected raster (EPSG:4326)
min_lon, min_lat, max_lon, max_lat = xx.ensemble_flood_extent.rio.bounds()
bounds = [[min_lat, min_lon], [max_lat, max_lon]]

def cumulative_mask_to_base64_png(masks_until_now):
    """Create an RGBA image (as base64 PNG) that accumulates binary masks with a fading-blue effect.
    Newer floods -> more intense blue and less transparency.
    """
    rgba = np.zeros((*masks_until_now[0].shape, 4), dtype=np.uint8)
    total = len(masks_until_now)

    for i, m in enumerate(masks_until_now):
        intensity = int(255 * (i / total))  # newer = stronger blue
        alpha = int(180 * (i / total))      # older = more transparent
        rgba[..., 2] += m.astype(np.uint8) * intensity
        rgba[..., 3] = np.maximum(rgba[..., 3], m.astype(np.uint8) * alpha)

    rgba[..., 2] = np.clip(rgba[..., 2], 0, 255)
    rgba[..., 3] = np.clip(rgba[..., 3], 0, 255)

    img = Image.fromarray(rgba)
    with io.BytesIO() as buf:
        img.save(buf, format='PNG')
        return "data:image/png;base64," + base64.b64encode(buf.getvalue()).decode()

# Generate cumulative overlays (as data URLs)
overlay_data_urls = []
for i in range(1, len(binary_masks) + 1):
    subset = binary_masks[:i]
    overlay_data_urls.append(cumulative_mask_to_base64_png(subset))

print(f"Prepared {len(overlay_data_urls)} cumulative overlays.")


Prepared 14 cumulative overlays.


## 8) High-quality GIF/MP4 export (OSM XYZ -> reproject once -> map + chart + progress bar)

This block:
- Downloads OSM XYZ tiles covering the AOI at a chosen zoom, stitches them, and reprojects to **EPSG:4326** and to your raster grid size.
- Replays the flood overlays with the same **cumulative fade** logic as the viewer.
- Draws a **large bar chart** (km² per day) and a **static legend** that **does not flicker**.
- Exports **GIF** and (if `ffmpeg` is available) **MP4**.



In [15]:
# === VIDEO EXPORT: OSM XYZ -> reprojection -> cumulative flood fade + big bar chart + progress bar ===
# Allow large PNGs and silence decompression warnings
Image.MAX_IMAGE_PIXELS = None
warnings.simplefilter("ignore", Image.DecompressionBombWarning)

# ---------- Parameters ----------
overlay_dir = FLOOD_DIR
ZOOM = 9
CHART_WIDTH_FRACTION = 0.45
TARGET_OUT_WIDTH = 1920
USER_AGENT = "Mozilla/5.0 (video-export; OSM tiles via requests)"
TIMEOUT = 10

_scale = TARGET_OUT_WIDTH / 1920.0
TITLE_SIZE      = int(20 * _scale)
AXIS_LABEL_SIZE = int(14 * _scale)
TICK_SIZE       = int(12 * _scale)
VALUE_SIZE      = int(12 * _scale)
MAP_TITLE_PX = int(26 * _scale)
MAP_LABEL_PX = int(20 * _scale)
MAP_DATE_PX  = int(22 * _scale)

# ---------- Helpers ----------
def _font(px):
    for p in ["DejaVuSans.ttf","/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",
              "/System/Library/Fonts/Supplemental/Arial Unicode.ttf","/Library/Fonts/Arial.ttf"]:
        try: return ImageFont.truetype(p, px)
        except: pass
    return ImageFont.load_default()

def _label_to_date(s):
    m = re.search(r'\d{4}-\d{2}-\d{2}', str(s))
    if m:
        try: return datetime.strptime(m.group(0), "%Y-%m-%d").date()
        except: pass
    try:
        return getattr(s, 'astype')('datetime64[D]').astype('datetime64[ms]').tolist().date()
    except:
        return None

def _build_range_text(labels_):
    ds = [d for d in (_label_to_date(x) for x in labels_) if d is not None]
    if not ds: return "Flood period"
    d1, d2 = min(ds), max(ds)
    return f"{d1.day}.{d1.month} - {d2.day:02d}.{d2.month}.{d2.year}"

def _format_frame_date(label):
    d = _label_to_date(label)
    return f"{d.day}.{d.month}.{d.year}" if d else str(label)

def _pick_xticks(n, target=8):
    if n <= 1: return [0]
    step = max(1, round(n / target))
    idx = list(range(0, n, step))
    if (n-1) not in idx: idx.append(n-1)
    return sorted(set(idx))

def _nice_upper(x):
    if x <= 0: return 1.0
    k = 10 ** math.floor(math.log10(x))
    for m in [1, 2, 2.5, 5, 10]:
        if m * k >= x * 1.05: return m * k
    return 10 * k

def _pad_even_rgb(arr):
    # FFmpeg (yuv420p) needs even dimensions -> pad 1px if needed.
    h, w = arr.shape[:2]
    ph = h % 2
    pw = w % 2
    if ph or pw:
        arr = np.pad(arr, ((0, ph), (0, pw), (0, 0)), mode="edge")
    return arr

# ---------- OSM XYZ ----------
ORIGIN = 20037508.342789244
TILE   = 256

def _latlon_to_tile(lat, lon, z):
    lat_rad = math.radians(lat); n = 2.0 ** z
    xtile = int((lon + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.log(math.tan(lat_rad) + 1.0 / math.cos(lat_rad)) / math.pi) / 2.0 * n)
    return xtile, ytile

def _tile_bounds_3857(x, y, z):
    n = 2 ** z; ts = (2 * ORIGIN) / n
    xmin = -ORIGIN + x * ts; xmax = xmin + ts; ymax = ORIGIN - y * ts; ymin = ymax - ts
    return xmin, ymin, xmax, ymax, ts

def _fetch_osm_xyz_mosaic(bounds_4326, z):
    (min_lat, min_lon), (max_lat, max_lon) = bounds_4326
    x0, y1 = _latlon_to_tile(min_lat, min_lon, z); x1, y0 = _latlon_to_tile(max_lat, max_lon, z)
    if x0 > x1: x0, x1 = x1, x0
    if y0 > y1: y0, y1 = y1, y0
    cols = x1 - x0 + 1; rows = y1 - y0 + 1
    mosaic = Image.new("RGB", (cols*TILE, rows*TILE), (255,255,255))
    session = requests.Session(); session.headers.update({"User-Agent": USER_AGENT})
    subs = ["a","b","c"]; k=0
    for yy in range(y0, y1+1):
        for xx in range(x0, x1+1):
            sub = subs[k%3]; k+=1
            url = f"https://{sub}.tile.openstreetmap.org/{z}/{xx}/{yy}.png"
            try:
                r = session.get(url, timeout=TIMEOUT); r.raise_for_status()
                tile = Image.open(io.BytesIO(r.content)).convert("RGB")
            except Exception:
                tile = Image.new("RGB", (TILE, TILE), (255,255,255))
            mosaic.paste(tile, ((xx-x0)*TILE, (yy-y0)*TILE))
    xmin, ymin, xmax, ymax, ts = _tile_bounds_3857(x0, y0, z)
    px = ts / TILE
    transform = Affine(px, 0, xmin, 0, -px, ymax)
    return np.array(mosaic, dtype=np.uint8), transform

def _osm_to_overlay_grid(bounds_4326, H, W, z):
    rgb_merc, src_transform = _fetch_osm_xyz_mosaic(bounds_4326, z)
    src_crs = rasterio.crs.CRS.from_epsg(3857); dst_crs = rasterio.crs.CRS.from_epsg(4326)
    (min_lat, min_lon), (max_lat, max_lon) = bounds_4326
    west, south, east, north = float(min_lon), float(min_lat), float(max_lon), float(max_lat)
    dst_transform = from_bounds(west, south, east, north, W, H)
    dst = np.zeros((3, H, W), dtype=np.uint8)
    for b in range(3):
        reproject(source=rgb_merc[:, :, b], destination=dst[b],
                  src_transform=src_transform, src_crs=src_crs,
                  dst_transform=dst_transform, dst_crs=dst_crs,
                  resampling=Resampling.bilinear)
    return np.transpose(dst, (1, 2, 0))

# ---------- Read overlays & sizes ----------
if 'bounds' not in globals():
    raise RuntimeError("Missing bounds = [[min_lat,min_lon],[max_lat,max_lon]] (EPSG:4326).")

overlay_files = sorted([f for f in os.listdir(overlay_dir) if f.lower().endswith(".png")])
if not overlay_files:
    raise RuntimeError(f"No PNG files found in '{overlay_dir}'.")

arr0 = np.array(Image.open(os.path.join(overlay_dir, overlay_files[0])).convert("RGBA"))
H0, W0 = arr0.shape[:2]

map_w   = int(TARGET_OUT_WIDTH / (1.0 + CHART_WIDTH_FRACTION))
chart_w = TARGET_OUT_WIDTH - map_w
map_h   = int(round(map_w * (H0 / W0)))   # keep aspect ratio
if map_h % 2:  # enforce even height for H.264
    map_h += 1
out_w, out_h = TARGET_OUT_WIDTH, map_h

labels_video = [os.path.splitext(f)[0] for f in overlay_files]

# ---------- Pixel area weighting (km^2 per row at given lat) ----------
(min_lat, min_lon), (max_lat, max_lon) = bounds
dlon_rad = math.radians((max_lon - min_lon) / W0)
dlat     = (max_lat - min_lat) / H0
phi_top    = np.radians(max_lat - np.arange(0, H0)   * dlat)
phi_bottom = np.radians(max_lat - np.arange(1, H0+1) * dlat)
row_area_m2 = (6378137.0 ** 2) * dlon_rad * np.abs(np.sin(phi_top) - np.sin(phi_bottom))

areas_km2 = []
resized_masks = []  # downscaled masks to (map_h, map_w)
for f in overlay_files:
    A = np.array(Image.open(os.path.join(overlay_dir, f)).convert("RGBA"))
    mask_full = (A[...,2] > 0) & (A[...,3] > 0)

    counts = mask_full.sum(axis=1).astype(np.float64)
    areas_km2.append(float(np.dot(counts, row_area_m2)) / 1e6)

    m_img = Image.fromarray(mask_full.astype(np.uint8)*255, mode="L").resize((map_w, map_h), Image.NEAREST)
    resized_masks.append(np.array(m_img, dtype=np.uint8) > 0)

areas_km2 = np.array(areas_km2, dtype=np.float64)
N = len(resized_masks)

# ---------- Basemap to target grid ----------
basemap_rgb = _osm_to_overlay_grid(bounds, map_h, map_w, ZOOM)

# ---------- Bar chart (pre-render per frame) ----------
def _short_label(lbl):
    d = _label_to_date(lbl)
    return f"{d.day}.{d.month}" if d else str(lbl)

y_max = _nice_upper(float(np.max(areas_km2)))
tick_idx    = _pick_xticks(N, target=8)
tick_labels = [_short_label(lbl) for lbl in labels_video]

def _make_bar_chart_img(areas_km2, highlight_idx, width_px, height_px, y_max, tick_idx, tick_labels):
    fig = plt.figure(figsize=(width_px/100, height_px/100), dpi=140)
    ax  = fig.add_subplot(111)

    # --- BAR CHART (nezmenené, len kozmetika) ---
    x = np.arange(len(areas_km2))
    bars = ax.bar(x, areas_km2, width=0.85, linewidth=0)

    for j, b in enumerate(bars):
        if j == highlight_idx:
            b.set_color("#1f77b4"); b.set_alpha(0.95); b.set_edgecolor("#0d2742"); b.set_linewidth(1.0)
        else:
            b.set_color("#e6e6e6"); b.set_alpha(1.0)

    ax.text(highlight_idx, areas_km2[highlight_idx] * 1.01, f"{areas_km2[highlight_idx]:,.0f}",
            ha="center", va="bottom", fontsize=VALUE_SIZE, fontweight="bold")

    ax.set_title("Flooded area (km²)", pad=12, fontsize=TITLE_SIZE, weight="bold")
    ax.set_ylabel("km²", fontsize=AXIS_LABEL_SIZE)
    ax.tick_params(axis="both", labelsize=TICK_SIZE)

    ax.set_ylim(0, y_max)
    ax.yaxis.set_major_locator(mticker.MaxNLocator(nbins=5))
    ax.yaxis.set_major_formatter(mticker.StrMethodFormatter("{x:,.0f}"))

    ax.set_xticks(tick_idx)
    ax.set_xticklabels([tick_labels[t] for t in tick_idx], rotation=45, ha="right", fontsize=TICK_SIZE)

    for s in ["top","right","left","bottom"]: ax.spines[s].set_visible(False)
    ax.grid(axis="y", linestyle=(0,(2,3)), linewidth=1.2, alpha=0.6)

    # --- PROGRESS BAR "FLOOD METER" (gradient + ikonka 🌊) ---
    N = len(areas_km2)
    curr = highlight_idx + 1  # aktuálny krok (1..N)

    pb_ax = fig.add_axes([0.12, 0.10, 0.86, 0.06])   # [left, bottom, width, height] v figure frac
    pb_ax.set_xlim(0, N); pb_ax.set_ylim(0, 1); pb_ax.axis("off")

    # podklad: jemná šedá s oblými rohmi
    bg = FancyBboxPatch(
        (0, 0.2), N, 0.6,
        boxstyle="round,pad=0.15,rounding_size=0.25",
        linewidth=0, facecolor="#eeeeee"
    )
    pb_ax.add_patch(bg)

    # gradient (svetlomodrá -> tmavomodrá), klipnutý na aktuálny progress
    grad = np.linspace(0, 1, 512).reshape(1, -1)             # 1 x 512
    im = pb_ax.imshow(
        grad,
        extent=[0, curr, 0.2, 0.8],                          # šírka = aktuálny progress
        cmap=plt.get_cmap("Blues"),
        aspect="auto",
        zorder=2
    )
    clip = FancyBboxPatch(
        (0, 0.2), curr, 0.6,
        boxstyle="round,pad=0.15,rounding_size=0.25",
        linewidth=0, facecolor="none"
    )
    pb_ax.add_patch(clip)
    im.set_clip_path(clip)

    # tenký tmavší „okraj“ na vrchu progressu (vizuálny lesk)
    gloss = FancyBboxPatch(
        (0, 0.68), curr, 0.06,
        boxstyle="round,pad=0.12,rounding_size=0.25",
        linewidth=0, facecolor=(1,1,1,0.25), zorder=3
    )
    pb_ax.add_patch(gloss)

    # --- layout a export do obrazu ---
    fig.subplots_adjust(left=0.18, right=0.98, top=0.86, bottom=0.50)
    fig.patch.set_facecolor("white"); ax.set_facecolor("white")

    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=140)
    plt.close(fig)
    buf.seek(0)
    im = Image.open(buf).convert("RGB").resize((width_px, height_px), Image.BICUBIC)
    return np.array(im, dtype=np.uint8)


chart_imgs = [_make_bar_chart_img(areas_km2, i, chart_w, map_h, y_max, tick_idx, tick_labels) for i in range(N)]

# ---------- Static, opaque legend (no flicker) ----------
def _make_legend_tile(range_text, W, H, legend_max_frac=(0.20, 0.12)):
    f_title = _font(MAP_TITLE_PX)
    f_label = _font(MAP_LABEL_PX)
    pad  = max(6, int(min(W,H)*0.015))
    gap  = max(4, pad//2)
    sw = sh = max(18, int(min(W,H)*0.02))

    max_w = int(W * legend_max_frac[0])
    max_h = int(H * legend_max_frac[1])

    tmp = Image.new("RGBA", (1,1))
    dtmp = ImageDraw.Draw(tmp)
    def _measure(dr, txt, f):
        if hasattr(dr, "textbbox"):
            l,t,r,b = dr.textbbox((0,0), txt, font=f); return (r-l, b-t)
        return dr.textsize(txt, font=f)

    twt, tht     = _measure(dtmp, range_text, f_title)
    w_new, h_new = _measure(dtmp, "Newer floods", f_label)
    w_old, h_old = _measure(dtmp, "Older floods", f_label)
    text_w = max(w_new, w_old)

    inner_w = max(twt, sw + gap + text_w)
    inner_h = tht + gap + sh + gap + sh
    box_w = min(pad*2 + inner_w, max_w)
    box_h = min(pad*2 + inner_h, max_h)

    # Fully opaque (alpha=255) -> no flicker
    tile = Image.new("RGBA", (box_w, box_h), (255,255,255,255))
    dr = ImageDraw.Draw(tile)

    tx = pad; ty = pad//2
    dr.text((tx, ty), range_text, fill=(0,0,0), font=f_title)

    y1 = ty + tht + gap*2 
    dr.rectangle((tx, y1, tx + sw, y1 + sh), fill=(180,200,255))
    dr.text((tx + sw + gap, y1 + (sh - h_new)//2), "Older floods", fill=(0,0,0), font=f_label)

    y2 = y1 + sh + gap
    dr.rectangle((tx, y2, tx + sw, y2 + sh), fill=(0,70,255))
    dr.text((tx + sw + gap, y2 + (sh - h_new)//2), "Newer floods", fill=(0,0,0), font=f_label)

    pos = (pad, H - box_h - pad)   # bottom-left
    return tile, pos

def _draw_date_only(pil_img, date_text):
    if not date_text: return
    f = _font(MAP_DATE_PX)
    dr = ImageDraw.Draw(pil_img)
    W, H = pil_img.size
    pad = max(6, int(min(W, H) * 0.02))
    t = str(date_text)
    if hasattr(dr,"textbbox"):
        x, y = W - pad, H - pad
        x0,y0,x1,y1 = dr.textbbox((x, y), t, font=f, anchor="rb")
        dr.rectangle((x0-6, y0-4, x1+6, y1+4), fill=(255,255,255))
        dr.text((x, y), t, fill=(0,0,0), font=f, anchor="rb")
    else:
        tw, th = dr.textsize(t, font=f)
        x = W - pad - tw; y = H - pad - th
        dr.rectangle((x-6, y-4, x+tw+6, y+th+4), fill=(255,255,255))
        dr.text((x, y), t, fill=(0,0,0), font=f)

# ---------- Build frames ----------
range_text = _build_range_text(labels_video)
LEG_TILE, LEG_POS = _make_legend_tile(range_text, map_w, map_h)

blue_sum  = np.zeros((map_h, map_w), dtype=np.uint32)  # sum of indices where flood occurred
last_day  = np.zeros((map_h, map_w), dtype=np.uint16)  # last frame index with flood

frames = []
for i, msk in enumerate(resized_masks, start=1):
    if msk.any():
        blue_sum[msk] += i
        last_day[msk]  = i

    blue  = (255.0 * (blue_sum / float(i))).astype(np.float32)
    alpha = (180.0 * (last_day / float(i))).astype(np.float32)

    ov = np.zeros((map_h, map_w, 4), dtype=np.uint8)
    ov[..., 2] = np.clip(blue,  0, 255).astype(np.uint8)   # Blue
    ov[..., 3] = np.clip(alpha, 0, 255).astype(np.uint8)   # Alpha

    bg = Image.fromarray(basemap_rgb.copy(), mode="RGB")
    ov_img = Image.fromarray(ov, mode="RGBA")
    bg.paste(ov_img, mask=ov_img.split()[3])

    # static legend + date
    bg.paste(LEG_TILE, LEG_POS, LEG_TILE)
    frame_date = _format_frame_date(labels_video[i-1])
    _draw_date_only(bg, frame_date)

    # ---- Title (centered) ----
    draw = ImageDraw.Draw(bg)
    title_text = "GFM cumulative floods"
    font_title = _font(28)

    if hasattr(draw, "textbbox"):
        x0, y0, x1, y1 = draw.textbbox((0, 0), title_text, font=font_title)
        text_w, text_h = (x1 - x0), (y1 - y0)
    else:
        bx0, by0, bx1, by1 = font_title.getbbox(title_text)
        text_w, text_h = (bx1 - bx0), (by1 - by0)

    draw.text(
        ((bg.width - text_w) // 2, 10),  # centrovanie
        title_text,
        font=font_title,
        fill=(0, 0, 0)
    )

    # stitch [ MAP | CHART ]
        # ---- stitch [ MAP | CHART ] with smooth fade ----
    canvas = np.full((map_h, out_w, 3), 255, dtype=np.uint8)  # biele pozadie pre graf
    map_arr = np.array(bg, dtype=np.uint8)

    # koľko z mapy sa má plynulo rozplynúť do bieleho pozadia (15 % šírky mapy)
    fade_width = max(8, int(map_w * 0.05))

    # horizontálny gradient 1 -> 0 cez fade zónu (šírka = fade_width, výška = map_h)
    grad = np.linspace(1.0, 0.0, fade_width, dtype=np.float32)[None, :, None]
    grad = np.repeat(grad, map_h, axis=0)  # (map_h, fade_width, 1)

    # časť mapy bez fade
    left_part = map_arr[:, :map_w - fade_width, :]

    # plynulý prechod mapy do bielej
    right_fade_src = map_arr[:, map_w - fade_width:map_w, :].astype(np.float32)
    right_fade = (right_fade_src * grad + 255.0 * (1.0 - grad)).astype(np.uint8)

    # zloženie do canvasu
    canvas[:, :map_w - fade_width, :] = left_part
    canvas[:, map_w - fade_width:map_w, :] = right_fade
    canvas[:, map_w:, :] = chart_imgs[i-1]

    frames.append(_pad_even_rgb(canvas))

fps = 3.0
print(f"✅ Frames: {len(frames)}, size={frames[0].shape}, fps={fps:.2f}")

# ---------- Export ----------
gif_path="flood_map_chart.gif"
imageio.mimsave(gif_path, frames, duration=1.0/fps, loop=0)
print(f"🎞️ GIF: {gif_path}")

mp4_path="flood_map_chart.mp4"
if shutil.which("ffmpeg"):
    try:
        with imageio.get_writer(
            mp4_path, fps=float(fps), codec="libx264", macro_block_size=None,
            ffmpeg_params=["-loglevel","error","-movflags","+faststart","-pix_fmt","yuv420p","-probesize","50M"]
        ) as w:
            for fr in frames: w.append_data(fr)
        print(f"🎬 MP4: {mp4_path}")
    except Exception as e:
        print(f"⚠️ Direct MP4 failed: {e} -> re-encode from GIF")
        subprocess.run([
            "ffmpeg","-y","-loglevel","error","-i",gif_path,
            "-vf","scale=trunc(iw/2)*2:trunc(ih/2)*2",
            "-c:v","libx264","-pix_fmt","yuv420p","-movflags","+faststart", mp4_path
        ], check=True)
        print(f"🎬 MP4 (from GIF): {mp4_path}")
else:
    print("ℹ️ ffmpeg not found -> Skipping MP4. Convert GIF later with ffmpeg if needed.")


✅ Frames: 67, size=(1110, 1920, 3), fps=3.00
🎞️ GIF: flood_map_chart.gif
🎬 MP4: flood_map_chart.mp4


In [ ]:
 from IPython.display import Image as IPImage, display

gif_path = "flood_map_chart.gif"
display(IPImage(filename=gif_path)) 



🌍 [Click here for GIF animation](https://martinmib-ba.github.io/GFM/docs/flood_map_chart.gif)

🗺️ [Click here to open the map](https://martinmib-ba.github.io/GFM/docs/flood_map_chart.html)
